# **Optimizacija fazi kontrolera za vožnju automobila korišćenjem genetskog algoritma**

## **Uvod**

Cilj ovog rada je bila optimizacija fazi kontrolera za vožnju korišćenjem genetskog algoritma. Kontroler je upravljao automobilom koji se kretao po otvorenom ili zatvorenom putu, u zavisnosti od zadatih opcija.  

U radu je prvo pokazano da je moguće konstruisati fazi kontroler koji će pravilno upravljati automobilom, a zatim je pokazano i da je moguće optimizovati funkcije pripadnosti ulaza i izlaza, na osnovu zadatih pravila uz pomoć genetskog algoritma, tako da pravilno upravlja automobilom.  

Uspešnost genetskog algoritma je merena tako što se optimizovani sistem upoređivao sa ručno napravljenim sistemom.

Automobilom se upravlja pravilno ako se automobil ne sudara sa ivicama puta i ako se kreće po sredini puta, koliko god je to moguće.

Automobil je testiran na dve vrste puta. Prvi tip puta je krivudav put koji je izgenerisan sinusiodom, a drugi tip puta je bio zatvoren kružni put izgenerisan pomoću konveksnog omotača nasumično odabranog skupa koordinata.

**Reference:**
> Fazi Logika: 
- http://poincare.matf.bg.ac.rs/~stefan/ri/3.html
- http://www.dma.fi.upm.es/recursos/aplicaciones/logica_borrosa/web/fuzzy_inferencia/mamdani2_en.htm

> Automatizacija vozila preko fazi kontrolera:
- http://www.ijarset.com/upload/2018/march/11-IJARSET-Vipul-1.pdf

> Optimizacija fazi sistema preko genetskog Algoritma: 
- https://www.researchgate.net/publication/261464574_Optimization_of_a_fuzzy_logic_controller_using_genetic_algorithms

## **Generisanje puta**

Za testiranje algoritma su korišćene 2 vrste puta:
- Konstantan krivudav put generisan sinusnom funkcijom
- Nasumično generisan put korišćenjem convex hull algoritma

Radi bržeg pokretanja programa, putevi se unapred mogu izgenerisati pokretanjem `display_matrix_generator.py` koji će snimiti matrice za obe vrste puta.

**Generisanje krivudavog puta:**

In [0]:
def generate_sin_path():
    coords1 = []
    coords2 = []
    offset = 200
    num_of_points = 20

    for i in range(constants.SCREEN_WIDTH):
        coords1.append((i, math.sin(0.01*i) * 200 + 250))

    coords2 = lt.apply_translation(0, offset, coords1)
    coords2.reverse() #? in order to properly match coords for polygon

    polygon = coords1 + coords2
    
    is_closed = False
    return polygon, is_closed

**Generisanje konveksnog poligona:**

In [0]:
def generate_convex_polygon(): 
    #? Guarantees that we will have car in our convex polygon
    coords = [(constants.CAR_POS_X, constants.CAR_POS_Y + 10)]
    num_of_points = 20
    
    for i in range (num_of_points):
        x = random.randrange(constants.SCREEN_WIDTH)
        y = random.randrange(constants.SCREEN_HEIGHT)
        coords.append((x, y))

    coords1 = ch.gift_wrap(coords)

    scaling_factor = 0.6
    offset_x = 200
    offset_y = 150
    coords2 = lt.apply_scaling(scaling_factor, scaling_factor, coords1)
    coords2 = lt.apply_translation(offset_x, offset_y, coords2)

    polygon = [coords1, coords2]
    
    is_closed = True
    return polygon, is_closed

#### **Krivudav put**

![Krivudav put](https://i.imgur.com/ebntYBh.png)

#### **Zatvoren put**

![Zatvoren put](https://i.imgur.com/hWVpZ59.png)


### **Senzori: Uvod**

Senzori se koriste u simulaciji da izračunaju rastojanje između vozila i krajeva puta. Rastojanja se koriste kao ulaz za fazi sistem. Svi senzori se u ovom slučaju nalaze sa prednje strane vozila. Razlikujemo tri vrste senzora:
- **levi senzor**: računa rastojanje između vozila i kraja puta koji se nalazi levo od vozila;
- **prednji senzor**: računa rastojanje između vozila i kraja puta koji se nalazi ispred vozila;
- **desni senzor**: računa rastojanje između vozila i kraja puta koji se nalazi desno od vozila;


![senzori1](https://i.imgur.com/ItnfUpG.png)

### **Senzori: Presečna tačka**
Da bi izračunali dužinu linije senzora, potrebno je da pronađemo tačku u bitovskoj mapi gde se senzorna prava preseca put. Ako fiksiramo rotaciju vozila, onda je prednji senzor paralelan sa x-osom, a levi i desni senzori su paralelni sa y-osom. Bas kao i na slici, možemo iterirati po x i y koordinatama bitovske mape dok ne stignemo do presečne tačke. Koristimo sličnu ideju i kad rotacije vozila nije fiksirana.

- Neka je r = (Xo, Yo)** vektor pozicije vozila, **a** ugao rotacije vozila i neka **z** krajnja tača senzora u sistemu sa reperom u tački (Xo, Yo), gde je **|z|** intenzitet vektora;
- Ugao vektora **z** za prednji senzor je jednak uglu rotacije vozila tj. **a**: **z = (|z|·cos(a), |z|·sin(a)) = (z·cos(a), z·sin(a));**
- Pozicija krajnje tačke senzora **S(z)** je jednaka **r * z**;
- Mozemo povećavati intenzitet vektor **z** dok tačka **S** ne postane presečna tačka;
- U svakoj iteraciji računamo **S(z) kao: **s(z) = r + z = (Xo + z·cos(a), Yo + z·sin(a)).**
	
![senzori2](https://i.imgur.com/FIXfZrE.png)
- Rastojanja za levi i desni senzor računamo na isti način. Dovoljno je da uvećamo ugao **a** za **90°** odnosno smanjimo za **90°** stepeni

In [0]:
def sensor(self, name, screen, angle_direction):
        angle = -self.angle/180*math.pi
        pos_x = self.center_position().x
        pos_y = self.center_position().y
        z = 0
        while(valid_position(int(pos_x), int(pos_y)) and screen.get_at((int(pos_x), int(pos_y))) == constants.PATH_COLOR):
            z = z + 1
            pos_x = self.center_position().x + z*math.cos(angle - angle_direction)
            pos_y = self.center_position().y + z*math.sin(angle - angle_direction)
            
        sensor_input = str(distance(self.center_position().x, self.center_position().y, pos_x, pos_y))
        return sensor_input

## **Senzori: Optimizacija**

Pošto svaka jedinka prelazi sličnu putanju, neke pozicije se računaju više puta. Računanje rastojanja je skupa operacija,  Ne pravimo veliku grešku ako poziciju vozila i ugao rotacije vozila zaokružimo na ceo broj. U tom slučaju imamo konačan broj stanja u koje vozilo može da dođe. U mapi možemo čuvati ključ koji predstavlja uređenu trojku (**Xo, Yo, a)** tj. poziciju vozila i ugao. Taj ključ se preslikava u uredjenu trojku **(left_sensor_input, front_sensor_input, right_sensor_input)**. Ovom optimizacijom izbacujemo *dupla* izračunavanja. Posledica je spora inicijalizacija populacije i sporo računanje prvih par generacija.

In [0]:
def get_sensors(car, road_matrix, memory):
    car_x = int(car.center_position().x)
    car_y = int(car.center_position().y)
    angle = int(car.angle)
    if (car_x, car_y, angle) in memory:
        return memory[(car_x, car_y, angle)]
    left, front, right = car.get_sensors2(road_matrix)
    memory[(car_x, car_y, angle)] = (left, front, right)
    return left, front, right

## **Fazi sistem: Uvod**


Fazi sistem se sastoji iz 3 ulazne celine (skup karakterističnih funkcija), 2 izlazne celine i nekoliko pravila. Ideja:


>Rastojanje vozila do kraja puta predstavljaju ulazne podatke za fazi sistem. Na osnovu ulaznih podataka računaju se karakteristične funkcije za svaku celinu (**fazifikacija**). Primenom pravila računamo karakteristične funkcije za svaku izlaznu celinu. Na kraju računamo izlazne podatke za svaku izlazni celinu kao centroid (**defazifikacija**).

> Sve karakteristične funkcije su trapeznog oblika, sem funkcija na kraju i na početku intervala koje su definisane kao 'ramp' funkcije. Bitna stvar koju treba napomenuti je da se mogu primetiti i funkcije koje su trougaonog oblika, ali se one u kodu zapravo predstavljaju kao trapezna funkcija kojoj se koordinate 2 viša temena poklapaju. Takođe, sve celine imaju interval na kom su definisane karakteristične funkcije. Smatramo da je pripadnost 0 van tog intervala.



## **Fazi Sistem: Implementacija ~ Mamdanijev fazi sistem**

### Klase:
- MFInput: Predstavlja jednu karakterističnu funkciju za ulazne podatke tj. računa pripadnost na osnovu ulaza. 
- MFOutput: Predstavlja jednu karakterističnu funkciju za izlazne podatke tj. računa pripadnost na osnovu definisanih pravila.
- Rule: Predstavlja jedno pravilo fazi sistema.
- FuzzyInput: Celina koja predstavlja skup karakterističnih funkcija za ulaz (MFInput). 
- FuzzyOutput: Celina koja predstavlja skup karakterističnih funkcija za izlaz (MFOutput).
- FuzzySystem: Kompletan fazi sistem koji se sastoji iz niza ulaznih celina (FuzzyInput), niza pravila (Rule), i jedne izlazne celine (FuzzyOutput). Pozivom metode *fit* simulira se Mamdanijev proces fazi sistema za dati ulaz (ulaz senzora). Vrednost izlaza se nalazi u atributu *solution*.

## **Fazi Sistem: Ulaz**

Postoje tri ulazne celine:
- `left_sensor`
- `right_sensor`
- `front_sensor`

Svaka od njih ima cetiri karakteristične funkcije (za svaku celinu posebno definisane):  
- `close`
- `midrange`
- `far`
- `very far`

One predstavljaju udaljenost od krajeva puta.   

>Primer optimizovanog rešenja sa manjim brojem iteracija i manjom populacijom jedinki za ulaz **(left, front, right) = (5, 30, 15)**:

![left_sensor](https://i.imgur.com/22ifsgu.png)
![front_sensor](https://i.imgur.com/czS16hX.png)
![right_sensor](https://i.imgur.com/4mEl0wU.png)

## **Fazi sistem: Izlaz**

Postoje dve izlazne celine:
- `velocity`
- `angle`

`velocity` označava brzinu, dok `angle` označava ugao skretanja volana.

`velocity` se sastoji od četiri karakteristične funkcije:
- `low`
- `middle`
- `high`
- `very high`

`angle` se sastoji od pet karakterističnih funkcija:
- `hard right`
- `right`
- `forward`
- `left`
- `hard left`

> Primer optimizovanog rešenja sa manjim brojem iteracija i manjom populacijom jedinki za ulaz **(left, front, right) = (5, 30, 15)**:

![angle](https://i.imgur.com/GCy71l1.png)
![velocity](https://i.imgur.com/pBpjum9.png)

> Parametri na osnovu kojih se generiše fazi sistem:


In [0]:
ALL_FUZZY_FUNCS = {
    "left_sensor": {
        "name": "left_sensor",
        "left_boundary": 0,
        "right_boundary": 50,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "right_sensor": {
        "name": "right_sensor",
        "left_boundary": 0,
        "right_boundary": 50,
        "mf_names": ["close", "midrange", "far", "very_far"],
        "is_input": True,
    },
    "front_sensor": {
        "name": "front_sensor",
        "left_boundary": 0,
        "right_boundary": 20,
        "mf_names": ["close", "midrange", "far", "very far"],
        "is_input": True,
    },
    "velocity": {
        "name": "velocity",
        "left_boundary": 1,
        "right_boundary": 20,
        "mf_names": ["low", "middle", "high", "very high"],
        "is_input": False,
    },
    "angle": {
        "name": "angle",
        "left_boundary": -45,
        "right_boundary": 45,
        "mf_names": ["hard right", "right", "forward", "left", "hard left"],
        "is_input": False,
    }
}

## **Fazi sistem: Ručno pravljen sistem**

> Ručno konstruisan fazi sistem koji uspešno upravlja automobilom:

In [0]:
import fuzzy

left = {
    "close"    : fuzzy.MFInput("close", np.array([0, 16]), np.array([1, 0])),
    "midrange" : fuzzy.MFInput("midrange", np.array([10, 14, 18, 24]), np.array([0, 1, 1, 0])),
    "far"      : fuzzy.MFInput("far", np.array([20, 36, 44]), np.array([0, 1, 0])),
    "very far" : fuzzy.MFInput("very far", np.array([32, 100]), np.array([0, 1]))
}

right = {
    "close"    : fuzzy.MFInput("close", np.array([0, 16]), np.array([1, 0])),
    "midrange" : fuzzy.MFInput("midrange", np.array([10, 16, 20, 28]), np.array([0, 1, 1, 0])),
    "far"      : fuzzy.MFInput("far", np.array([24, 40, 45]), np.array([0, 1, 0])),
    "very far" : fuzzy.MFInput("very far", np.array([35, 100]), np.array([0, 1]))
}

front = {
    "close"    : fuzzy.MFInput("close", np.array([0, 18]), np.array([1, 0])),
    "midrange" : fuzzy.MFInput("midrange", np.array([14, 20, 24, 28]), np.array([0, 1, 1, 0])),
    "far"      : fuzzy.MFInput("far", np.array([26, 34, 40]), np.array([0, 1, 0])),
    "very far" : fuzzy.MFInput("very far", np.array([35, 100]), np.array([0, 1])),
}

velocity = {
    "low"       : fuzzy.MFOutput("low", np.array([1, 2]), np.array([1, 0])),
    "middle"    : fuzzy.MFOutput("middle", np.array([1.5, 3, 3.5, 4]), np.array([0, 1, 1, 0])),
    "high"      : fuzzy.MFOutput("high", np.array([3.5, 4.5, 5.5]), np.array([0, 1, 0])),
    "very high" : fuzzy.MFOutput("very high", np.array([4, 8]), np.array([0, 1]))
}

angle = {
    "hard right" : fuzzy.MFOutput("hard right", np.array([25, 45]), np.array([1, 0])),
    "right"      : fuzzy.MFOutput("right", np.array([5, 22, 25]), np.array([0, 1, 0])),
    "forward"    : fuzzy.MFOutput("forward", np.array([-5, 0, 5]), np.array([0, 1, 0])),
    "left"       : fuzzy.MFOutput("left", np.array([-25, -22, -5]), np.array([0, 1, 0])),
    "hard left"  : fuzzy.MFOutput("hard left", np.array([-45, -25]), np.array([0, 1]))
}

left_sensor = fuzzy.FuzzyInput("left_sensor", np.array([
    left["close"],
    left["midrange"],
    left["far"],
    left["very far"]
]))

right_sensor = fuzzy.FuzzyInput("right_sensor", np.array([
    right["close"],
    right["midrange"],
    right["far"],
    right["very far"]
]))

front_sensor = fuzzy.FuzzyInput("front_sensor", np.array([
    front["close"],
    front["midrange"],
    front["far"],
    front["very far"] 
]))

f_velocity = fuzzy.FuzzyOutput("velocity", np.array([
    velocity["low"],
    velocity["middle"],
    velocity["high"],
    velocity["very high"]
]))

f_angle = fuzzy.FuzzyOutput("angle", np.array([
    angle["hard right"],
    angle["right"],
    angle["forward"],
    angle["left"],
    angle["hard left"]
]))

## **Fazi sistem: pravila**

Fazi pravila su ručno definisana uzimanjem nekih od postojećih kombinacija pravila za koje se činilo da imaju najviše smisla. 

Genetski algoritam je nezavistan od fazi pravila u smislu da kako god da definišemo pravila, genetski algoritam će naći neko dobro rešenje za tako definisana pravila. Razlozi zbog kojih ne ubacujemo pravila u optimizaciju genetskog algoritma:

> Već imamo neke informacije o tome kako treba da izgledaju pravila (npr. ako nam je desna strana blizu, pravilo koje kaže da skrenemo oštro desno očigledno nema smisla).

> Kombinatorna ekplozija. Za svaki ulazni argument u pravilu imamo *n+1* mogućnosti (ne moramo neku celinu iskoristiti kao argument) i za svaku izlaznu celinu imamo *n* mogućnosti. Za ugao imamo 625 mogućnosti, a za brzinu 500 mogućnosti. To je ukupno 312500 mogućnosti za pravila. Zajedno sa ulaznim i izlaznim celinama ovaj broj je ogroman, a evaluacija rešenja je skupa operacija. Praktično je neizvodljivo to izvršiti u našem slučaju ako uzmemo u obzir da vreme izvršavanja programa treba da bude relativno kratko.   
Druga opcija je da se optimizuju fazi pravila i fazi ulaz za fiksirani izlaz (u našoj implementaciji je lako prebaciti se na drugu opciju). Još jedna alternativa je da se fiksiraju i izlazne celine i pravila. 

Pravila koja su korišćena pri optimizaciji:

In [0]:
# Pravila za ugao
import fuzzy
import numpy as np

angle_rules = fuzzy.FuzzyRules(np.array([
    fuzzy.Rule(np.array([left["close"], right["midrange"]]), angle["hard right"]),
    fuzzy.Rule(np.array([left["midrange"], right["close"]]), angle["hard left"]),
    fuzzy.Rule(np.array([left["close"], right["far"]]), angle["hard right"]),
    fuzzy.Rule(np.array([left["far"], right["close"]]), angle["hard left"]),

    fuzzy.Rule(np.array([left["close"], front["close"]]), angle["hard right"]),
    fuzzy.Rule(np.array([right["close"], front["close"]]), angle["hard left"]),
    fuzzy.Rule(np.array([left["close"], front["midrange"]]), angle["right"]),
    fuzzy.Rule(np.array([right["close"], front["midrange"]]), angle["left"]),

    fuzzy.Rule(np.array([left["far"], right["midrange"]]), angle["hard left"]),
    fuzzy.Rule(np.array([left["midrange"], right["far"]]), angle["hard right"]),
    fuzzy.Rule(np.array([left["very far"], right["midrange"]]), angle["hard left"]),
    fuzzy.Rule(np.array([left["midrange"], right["very far"]]), angle["hard right"]),
    fuzzy.Rule(np.array([left["far"], right["close"]]), angle["left"]),
    fuzzy.Rule(np.array([left["close"], right["far"]]), angle["right"]),

    fuzzy.Rule(np.array([left["midrange"], right["midrange"]]), angle["forward"]),
    fuzzy.Rule(np.array([left["close"]]), angle["hard right"]),
    fuzzy.Rule(np.array([right["close"]]), angle["hard left"]),
]))

In [0]:
 # Pravila za brzinu
 
 velocity_rules = fuzzy.FuzzyRules(np.array([
        fuzzy.Rule(np.array([front["close"]]), velocity["low"]),
        fuzzy.Rule(np.array([front["midrange"]]), velocity["middle"]),
        fuzzy.Rule(np.array([front["far"]]), velocity["high"]),
        fuzzy.Rule(np.array([front["very far"]]), velocity["very high"])
    ]))

## **Genetski algoritam: Hromozom i početna populacija**

Svaki hromozom sadrži fazi sisteme za ugao i brzinu, kao i fitnes. Početni fazi sistemi se generišu pomoću `fuzzy_generator.py` fajla i inicijalizacija sistema je takva da garantuje valjanost.

#### **Generisanje funkcija pripadnosti kod početne populacije**

Najkritičniji deo algoritma je generisanje početne populacije, a jedini zaista nasumičan deo kod generisanja populacije je bio generisanje funkcija pripadnosti. Svaka funkcija pripadnosti je morala da poštuje sledeća ograničenja:
- Vrednosti za X su unutar dozvoljenog intervala
- Ne sme postojati nepokriven deo X ose unutar dozvoljenog intervala

Funkcija pripadnosti se sastojala iz 2 niza koordinata:
- Niz za X koji uzima vrednost iz intervala [`left_boundary`, `right_boundary`]
- Niz Y koji uzima jednu od vrednosti {0, 1}

Svaka trapezna funkcija je opisana sa 4 para tačaka koje označavaju kritične tačke trapeza. Slično, svaka *ramp* funkcija je opisana sa 2 para tačaka.

Y koordinate su bile generisane u skladu sa tim koju vrstu funkcije opisuju jer je poznato da će *ramp* funkcije biti samo na krajevima, dok će sve ostale funkcije biti trapezoidne. Takođe, bilo je poznato da prva *ramp* funkcija mora ići od 1 do 0 po Y osi, dok druga mora ići od 0 do 1. Y koordinate su bile izgenerisane sledećom funkcijom:

In [0]:
def get_ys(size):
    """
    generate 10|0110|...|0110|01 sequence of ys
    
    len(10|0110|...|0110|01) = size
    """
    ys = [1, 0]
    for i in range(2, size):
        #? if we are on the edges of the 0110
        #? here->0 110 or 011 0<-here
        if (i-2)%4 == 0 or (i-2)%4 == 3:
            ys.append(0)
        else:
            ys.append(1)
    return ys

X koordinate će biti izgenerisane tako da su sigurno unutar granica intervala, da sigurno pokrivaju X osu, i da se koordinate za različite funkcije pripadnosti sigurno ne seku na samoj X osi. Ideja:

> Izgenerišemo nekakav nasumičan skup početnih tačaka koji će sadržati levu i desnu granicu i gde će sve ostale tačke biti unutar granica.

> Odmah možemo da odredimo *ramp* funkcije, jer znamo da su one na krajevima intervala.

> Za ostale funkcije računamo središnje 2 tačke tako što ih nasumično biramo iz intervala koji sigurno uključuje trenutne 2 ključne tačke, i važi da se one ne nalaze na samom kraju tog intervala.

> Krajnje tačke računamo tako što levu tačku biramo tako da je sigurno manja od trenutne leve ključne tačke, dok je veća od prethodne. Za desnu tačku radimo slično, samo gledamo da je sigurno veća od trenutne desne ključne tačke, a da je manja od naredne.

Funkcija koja generiše X koordinate:


In [0]:
def get_xs(size, left, right):
    """
    Generates key points for trapezoidal functions.

    Guarantees that there are no uncovered areas of the x-axis
    """
    number_of_points = 4*(size-1)
    #? split interval to parts within key_points represented by the key_points array 
    key_points = random.sample(range(left+1, right-1), size-1)
    key_points.append(left)
    key_points.append(right)
    key_points.sort()

    xs = []
    #? left most figure (isn't a trapeziod)
    xs.append(key_points[0])
    xs.append(key_points[1])

    for i in range(1, size-1):
        #? choose middle points for the trapezoid
        new_xs = random.sample(range(key_points[i]-1, key_points[i+1]+1), 2)
        new_xs.sort()

        #? choose left-most edge so that is for sure 'leftest' point in the trapezoid 
        xs.append(random.randint(key_points[i-1]+1, key_points[i]))
        
        #? append middle points
        xs.append(new_xs[0])
        xs.append(new_xs[1])
        
        #? choose right-most edge so that is for sure 'rightest' point in the trapezoid
        xs.append(random.randint(key_points[i+1], key_points[i+2]-1))
    
    #? right most figure (isn't a trapeziod)
    xs.append(key_points[-2])
    xs.append(key_points[-1])

    return xs

![init](https://i.imgur.com/9jZlc6F.png)

Na osnovu ova dva niza X i Y se kontruiše početni nasumično generisan fazi sistem.


## **Genetski algoritam: Selekcija i elitizam**

Korišćena je turnirska selekcija u kombinaciji sa elitizmom. Elitizam je unapred uzimao određen broj najboljih jedinki, dok su ostale bile odabrane selekcijom:


In [0]:
TOURNAMENT_SIZE = 10

def create_group(population, group_size = TOURNAMENT_SIZE):
    ids = random.sample(range(0, population.size), group_size)
    return population[ids]

def select(population):
    # tournament selection
    group = create_group(population)
    result = group[0]
    for i in range(1, group.size):
        if group[i] < result:
            result = group[i]
    return result

## **Genetski algoritam: Ukrštanje**

Korišćeno je uniformno ukrštanje po funkcijama pripadnosti za svako od izlaznih pravila fazi sistema. Odvojeno su ukrštani delovi fazi sistema koji opisuju ugao i delovi fazi sistema koji opisuju brzinu:

In [0]:
def crossover(p1, p2):
    c1 = copy.deepcopy(p1)
    c2 = copy.deepcopy(p2)
    for i in range(c1.FSAngle.inputs.size):
        fuzzy_output1 = c1.FSAngle.inputs[i]
        fuzzy_output2 = c2.FSAngle.inputs[i]

        for j in range(fuzzy_output1.inputs.size):
            mf_output1 = fuzzy_output1[j]
            mf_output2 = fuzzy_output2[j]

            r = random.random()
            if r < 0.5:
                swap(mf_output1, mf_output2)

    for i in range(c1.FSVelocity.inputs.size):
        fuzzy_output1 = c1.FSVelocity.inputs[i]
        fuzzy_output2 = c2.FSVelocity.inputs[i]

        for j in range(fuzzy_output1.inputs.size):
            mf_output1 = fuzzy_output1[j]
            mf_output2 = fuzzy_output2[j]

            r = random.random()
            if r < 0.5:
                swap(mf_output1, mf_output2)
                
    return c1, c2

##  **Genetski algoritam: Mutacija**

Vršena je makro mutacija nad funkcijama pripadnosti koje određuju izlaze. Mutacija je podrazumevala generisanje novih vrednosti za kritične tačke na postojećoj funkciji koja je odabrana za mutaciju. Poštovana su sva ograničenja koje funkcija mora da zadovolji, slično kao i pri generisanju inicijalne populacije.

#### Mutacija trapezne funkcije

Nova funkcija je generisana tako što je svaka kritična tačka funkcije pripadnosti translirana po X osi za nasumičnu vrednost iz intervala [`-MUTATION_SPAN`, `MUTATION_SPAN`]. Zatim je vršena korekcija leve i desne granice, ako su izlazile van intervala.

#### Mutacija *ramp* funkcije
Nova funkcija je generisana tako što je samo leva, odnosno desna tačka funkcije bila pomerena za nasumičnu vrednost iz intervala [`-MUTATION_SPAN`, `MUTATION_SPAN`]. Zatim je vršena korekcija, slično kao i prilikom mutacije trapezne funkcije.


Uzimajući u obzir performanse programa, postoje 2 verovatnoće za mutaciju:
- `MUTATION_RATE`
- `MUTATION_GENOM_RATE`

`MUTATION_RATE` označava da li uopšte želimo da mutiramo trenutni hromozom, odnosno da li uopšte ulazimo u logiku za mutaciju.`MUTATION_GENOM_RATE` označava verovatnoću da mutiramo jednu od funkcija pripadnosti nakon što je odlučeno da se trenutni hromozom mutira:

In [0]:
def mutate(c, mutation_rate = MUTATION_RATE):
    r = random.random()
    if r > mutation_rate:
        return c

    for i in range(c.FSAngle.inputs.size):
        fuzzy_input = c.FSAngle.inputs[i]
        for j in range(fuzzy_input.inputs.size):
            mf_input = fuzzy_input[j]
            for k in range(fuzzy_input.inputs[j].size):
                
                r = random.random()
                if r < MUTATION_GENOM_RATE:
                    number_of_points = int(mf_input.size)
                    right_boundary = fuzzy_generator.ALL_FUZZY_FUNCS[fuzzy_input.name]["right_boundary"]
                    left_boundary = fuzzy_generator.ALL_FUZZY_FUNCS[fuzzy_input.name]["left_boundary"]

                    shift_value = random.randint(-MUTATION_SPAN, MUTATION_SPAN)

                    if number_of_points == 2 and mf_input.points[0][1] == 1:
                        new_value = mf_input.points[0][0] + shift_value
                        mf_input.points[0][0] = min(right_boundary, new_value)

                    elif number_of_points == 2 and mf_input.points[0][1] == 0:
                        new_value = mf_input.points[1][0] + shift_value
                        mf_input.points[1][0] = max(left_boundary, new_value)

                    else:
                        left = mf_input.points[0][0] + shift_value
                        shift_value = random.randint(-MUTATION_SPAN, MUTATION_SPAN)
                        right = mf_input.points[-1][0] + shift_value

                        if left < left_boundary:
                            left = left_boundary
                        
                        if right > right_boundary:
                            right = right_boundary
                        
                        if left > right:
                            swap(left, right_boundary)

                        if abs(left - right) <= 2:
                            right += 2 

                        shift_value = mf_input.points[1][0] + random.randint(-MUTATION_SPAN, MUTATION_SPAN)
                        mid_1 += shift_value
                        shift_value = mf_input.points[2][0] + random.randint(-MUTATION_SPAN, MUTATION_SPAN)
                        mid_2 += shift_value
                        
                        if mid_1 > mid_2:
                            mid_1, mid2 = swap(mid_1, mid_2)

                        new_xs = [left, mid_1, mid_2, right]

                        for t in range(4):
                            mf_input.points[t][0] = new_xs[t]
    return c

## **Genetski algoritam: Nedopustiva rešenja**

Prilikom testiranja programa primećeno je da se ponekad desi da se u trenutnoj populaciji nađe nedopustivo rešenje koje sadrži "rupu" u X osi.

Ovakva rešenja, iako nedopustiva, brzo su ispadala iz populacije jer su ili ostajala u mestu, ili su izletala sa puta.

## **Genetski algoritam: Funkcija evaluacije**

Fitnes jedinke se računaju tako što bi jedinka bila puštena da vozi po stazi sve dok jedan od sledećih uslova ne bude ispunjen:
- Jedinka je izletela sa puta
- Dostignut je maksimalan broj iteracija (jedinka nikada ne izleće van puta, pa mora nekako da prekine vožnju)
- Jedinka se ne pomera više od X iteracija

Kako je cilj jedinke da vozi što duže i što je moguće više po sredini puta fintes funkcija je računata kao:  

`left_right/iteration + punishment`

`left_right` meri koliko je jedinka išla po sredini puta, `iteration` označava koliko dugo je vozila, a `punishment` je penalizacija za one jedinke koje su prestale da se kreću, ali i dalje nisu izašle van puta i za jedinke koje su izašle van puta.

Ostaje mogućnost da se ubaci prosečna brzina kao jos jedan parametar za funkciju evaluacije, gde je ocena bolja što je prosečna ocena veća.

**Što je fitnes manji, jedinka je bolja**

In [0]:
def evaluate(FSAngle, FSVelocity, road_matrix, memory):
    """
    Runs a single simulation, movement params are calculated based on the fuzzy systems FSAngle and FSVelocity
    """
    car = vehicle.Car(constants.CAR_POS_X, constants.CAR_POS_Y, constants.CAR_ANGLE)

    iteration = 0
    past_pos = car.center_position()

    dec = decoder.Decoder(FSAngle, FSVelocity, car)
    dt = TIME_STEP

    total_distance = 0
    punishment = 0
    left_right = 0

    while iteration <= MAX_ITERATIONS:
        
        car.left_sensor_input, car.front_sensor_input, car.right_sensor_input = get_sensors(car, road_matrix, memory)
        ds, drot = dec.get_movement_params()
        car.update(dt, ds, drot)

        iteration += 1 
        total_distance += ds
        left_right += abs(float(car.left_sensor_input) - float(car.right_sensor_input))

        if iteration % 100 == 0:
            past_x, past_y = past_pos
            curr_x, curr_y = car.center_position()
            if vehicle.distance(past_x, past_y, curr_x, curr_y) < MIN_DISTANCE:
                break
            else:
                past_pos = car.center_position()

        if car.is_idle(iteration) or car.is_collided2(road_matrix):
            punishment = 150
            break

    return left_right/iteration + punishment

## **Genetski algoritam: Rezultati**

#### Parametri korišćeni pri treniranju:
- Veličina turnira: **5**
- Veličina populacije: **500**
- Procenat elitistički izabranih jedinki: **5%**
- Broj generacija: **20**
- Opseg mutacije: **2**
- Verotatnoća mutacije hromozoma: **0.1**
- Verovatnoća mutacije funkcije pripadnosti: **0.1**

#### Parametri korišćeni u fitnes funkciji:
- Maksimalan broj iteracija: **500**
- Minimalan put koje vozilo mora da pređe u **100 iteracija** da bi se smatralo pokretnim: **50**
- Penalizacija za izletanje sa puta ili za mirovanje: **150**

## **Optimizacija za krivudav put**

#### **Izgled funkcija pripadnosti za brzinu u ugao pre i posle treniranja**

##### **Ugao**

![Ugao pre treniranja](https://i.imgur.com/6QZXM1t.png)
![Ugao posle treniranja](https://i.imgur.com/vGHxSG7.png)

##### **Brzina**

![Brzina pre treniranja](https://i.imgur.com/OzMBaZm.png)
![Brzina posle treniranja](https://i.imgur.com/JaqU1kk.png)


#### Promena vrednosti fitnesa kroz generacije

![Promena fitnesa kroz generacije](https://i.postimg.cc/C1ShXjgd/Krivudavi-put.png)

### Optimizacija za zatvoren put

##### **Ugao**

![Ugao pre treniranja](https://i.imgur.com/UbcMGK2.png)
![Ugao posle treniranja](https://i.imgur.com/qVZosdV.png)

##### **Brzina**

![Brzina pre treniranja](https://i.imgur.com/CIvXUCc.png)
![Brzina posle treniranja](https://i.imgur.com/P12YPUW.png)


#### Promena vrednosti fitnesa kroz generacije

![Promena fitnesa kroz generacije](https://i.postimg.cc/2ydt1X97/Zatvoreni-put.png)


Sa grafika se primećuje da treniranje na krivudavom putu dalje bolje rezultate, dok se simulacijama pokazuje da daje i generalno bolju vožnju od treniranja na zatvorenom putu

## **Korišćenje optimizovanog fazi kontrolera na putevima za koje nisu trenirani**

Moguće je i korišćenje unapred treniranog fazi kontrolera na putu za koji fazi sistem nije bio treniran. Ovo je, u zavisnosti od puta, davalo različite rezultate.  
Najčešće neželjene situacije do kojih se dolazilo su bile nasumično menjanje smera, i vrtenje u krug. Bitna stvar koja je primećena je što je izuzetno retko dolazilo do izletanja automobila sa puta.

Pokazalo se da se jedinke trenirane na krivudavom putu bolje ponašaju od jedinki treniranih na zatvorenom putu kada se kreću po putu za koji nisu trenirane.


## **Zaključak**

U ovom radu je pokazano da je moguće optimizovati fazi kontroler za upravljanje automobilom uz pomoć genetskog algoritma.

Pokazano je da uz dovoljan broj iteracija automobil može dostići relativno pravilno kretanje po putu. U ovom slučaju su jedine prepreke krajevi puteva, ali pretpostavka je da fazi sistem može da se koristi za izbegavanje jednostavnijih prepreka. 

Takođe, pokazano je da je u nekim slučajevima moguće koristiti i kontrolere koji su trenirani na krivudavom putu za vožnju na zatvorenom putu, a pokazano je da može važiti i obrnuto.   
Uspešnost ovakve upotrebe kontrolera je najviše zavisila od toga na kakvom zatvorenom putu se vozilo treniralo, pošto je on bio nasumično generisan.

### **Dalji razvoj**

Dalji razvoj može ići u smeru vremenske optimizacije algoritma, sa posebnim akcentom na funkciju koja računa fitnes. Takođe, moguće je isprobati i trenirati algoritam na drugačijim konfiguracijama puta od onih koje su ovde predstavljene.

Fazi sistem bi mogao da se prevede na arduino čip kako bi upravljao stvarnim vozilom. 